In [1]:
import requests
from requests.structures import CaseInsensitiveDict

#url = "https://api.geoapify.com/v1/geocode/search?text=" + text + "&apiKey=3e1f56207e074f7a8108362ba0a7e5c5"

#headers = CaseInsensitiveDict()
#headers["Accept"] = "application/json"

#resp = requests.get(url, headers=headers)

#print(resp.status_code)

In [31]:
from pandas import json_normalize
addr = resp.json()

addr_df = json_normalize(addr)

#pd.DataFrame.from_dict(addr)

In [55]:
for i in addr_df['features']:
    for y in i:
        print(pd.DataFrame.from_dict([*y], y.values))

AttributeError: 'builtin_function_or_method' object has no attribute 'lower'

<built-in method values of dict object at 0x7f917616f200>


In [1]:
import pandas as pd
import geopandas as gpd
import json
from ast import literal_eval
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import json


GPEtags = pd.read_csv("20species_GPEtags.csv", sep = ";")

In [2]:
GPEtags = GPEtags.drop(columns = 'Unnamed: 0')

In [13]:
len(GPEtags)

4509

In [5]:
locs = []

for txt in GPEtags['GPE']:
    text = txt
    url = "https://api.geoapify.com/v1/geocode/search?text=" + text + "&apiKey=3e1f56207e074f7a8108362ba0a7e5c5"
    
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    resp = requests.get(url, headers=headers)
    locs.append(resp.json())
    

In [14]:
locs[:5]

[{'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {'datasource': {'sourcename': 'openstreetmap',
      'attribution': '© OpenStreetMap contributors',
      'license': 'Open Database License',
      'url': 'https://www.openstreetmap.org/copyright'},
     'country': 'India',
     'country_code': 'in',
     'lon': 78.6677428,
     'lat': 22.3511148,
     'formatted': 'India',
     'address_line1': 'India',
     'address_line2': '',
     'category': 'administrative',
     'timezone': {'name': 'Asia/Kolkata',
      'offset_STD': '+05:30',
      'offset_STD_seconds': 19800,
      'offset_DST': '+05:30',
      'offset_DST_seconds': 19800,
      'abbreviation_STD': 'IST',
      'abbreviation_DST': 'IST'},
     'result_type': 'country',
     'rank': {'importance': 0.957689135880987,
      'popularity': 0.7910727530603292,
      'confidence': 1,
      'match_type': 'full_match'},
     'place_id': '51ed084c4cbcaa5340593f24d7a8e2593640f00101f9014ca6040000000000c00

In [9]:
new = [x for x in locs if x != []]

In [6]:
test = pd.DataFrame(columns = ['name', 'geometry'])

for n in range(len(locs)):
    
    geom = {'geometry' : [locs[n]['features'][0]['properties']['lat'], locs[n]['features'][0]['properties']['lon']]}
    test.append(pd.DataFrame(geom))
    
    try:
        name = {'name' : [locs[n]['features'][0]['properties']['name']]}
    except:
        name = {'name' : [locs[n]['features'][0]['properties']['country']]}
    test.append(pd.DataFrame(name))

/tmp/ipykernel_96/2473999839.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test.append(pd.DataFrame(geom))
/tmp/ipykernel_96/2473999839.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test.append(pd.DataFrame(name))
/tmp/ipykernel_96/2473999839.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test.append(pd.DataFrame(geom))
/tmp/ipykernel_96/2473999839.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test.append(pd.DataFrame(name))
/tmp/ipykernel_96/2473999839.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test.append(pd.DataF

IndexError: list index out of range

In [10]:
test = pd.DataFrame(columns = ['name', 'geometry'])

for n in range(len(new)):
    
    geom = {'geometry' : [new[n]['features'][0]['properties']['lat'], new[n]['features'][0]['properties']['lon']]}
    test = pd.concat([test, pd.DataFrame(geom)])
    
    try:
        name = {'name' : [new[n]['features'][0]['properties']['name']]}
    except ValueError:
        name = {'name' : [new[n]['features'][0]['properties']['country']]}
    except:
        pass
    
    test = pd.concat([test, pd.DataFrame(name)])

IndexError: list index out of range

In [11]:
for n in range(len(new)):
    try:
        if new[n]['features'] == []:
             del new[n]
    except:
        if new[n] == []:
            del new[n]
    
    

IndexError: list index out of range

In [88]:
new = [x for x in locs if x != []]

In [102]:
len(locs)

4509

In [51]:
coord = [locs[21]['features'][0]['properties']['lat'], locs[21]['features'][0]['properties']['lon']]

In [8]:
try: 
    print(locs[0]['features'][0]['properties']['name'])
    
except:
    print(locs[0]['features'][0]['properties']['country'])

India
